<a href="https://colab.research.google.com/github/jinseriouspark/polaris-llm/blob/main/mistral_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n

Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install -U datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [3]:
from datasets import load_dataset
dataset = load_dataset('seriouspark/persona_1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

# Mistral Fine tuning

```
messages = [
{"role": "system", "content": "{system_message}"},
{"role": "user", "content": "{user_message}"},
{"role": "model", "content": "{model_messgage"}
]
```

In [4]:
def mistral_formatting(example):
  if example['input'] and len(example['input']) > 0:
    text = f"""[INST]user\n{example['instruction']}\nuser\n{example['input']}\nmodel\n{example['output']}[/INST]"""

  else:
    text = f"""[INST]user\n{example['instruction']}\nmodel\n{example['output']}[/INST]"""
  return {'prompt': text}


In [5]:
dataset = dataset.map(mistral_formatting)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 100
    })
})

# load model and tuning

In [7]:
!pip install -qU transformers==4.39.0 accelerate==0.27.1 bitsandbytes==0.42.0 peft==0.8.2 trl==0.7.10


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 22.6 MB/s eta 0:00:00


In [8]:
!pip install git+https://github.com/huggingface/trl.git@7630f877f91c556d9e5a3baa4b6e2894d90ff84c

  Cloning https://github.com/huggingface/trl.git (to revision 7630f877f91c556d9e5a3baa4b6e2894d90ff84c) to /tmp/pip-req-build-_5cuu0fs
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/trl.git /tmp/pip-req-build-_5cuu0fs
  Running command git rev-parse -q --verify 'sha^7630f877f91c556d9e5a3baa4b6e2894d90ff84c'
  Running command git fetch -q https://github.com/huggingface/trl.git 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Running command git checkout -q 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Resolved https://github.com/huggingface/trl.git to commit 7630f877f91c556d9e5a3baa4b6e2894d90ff84c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trl: filename=trl-0.7.12.dev0-py3-none-any.whl size=173433 sha256=fe544c140b7132ec7e1ab6754b836029bbdf39ca4aa2049df34e692c7c08700a
  Stored in directory: /root/.cache/pip/wheels/ad/f5/b1/f5ac48230936583c88cfde8

In [10]:
!nvidia-smi

Tue Jun  4 11:03:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [31]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time


from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)

from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

#logging.set_verbosity_error()

In [12]:
model_id = "mistralai/Mistral-7B-v0.3"

In [13]:
bnb_config = BitsAndBytesConfig(load_in_4bit = True,
                                 bnb_4bit_quant_type = 'nf4',
                                 bnb_4bit_compute_dtype = torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config,
                                             device_map = {"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [14]:
tokenizer.pad_token = tokenizer.eos_token

In [15]:
dataset = dataset.map(lambda samples : tokenizer(samples['prompt']), batched = True)
dataset = dataset['train'].train_test_split(test_size = 0.1)
dataset

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 90
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
})

In [16]:
train_data= dataset['train']
eval_data = dataset['test']
print(train_data[0])

{'instruction': '옥자 씨, 옥자 씨는 어떻게 하면 좋은 친구가 될 수 있을까요? 혹시 좋은 친구가 되기 위한 옥자 씨만의 노하우가 있으신가요?', 'input': '', 'output': '좋은 친구? 나는 친구들에게 진심으로 마음을 전하고, 힘이 되어주는 것이 중요하다고 생각해. 그리고 긍정적인 마음으로 친구들을 격려해주고, 함께 웃고 즐거운 시간을 보내는 것도 중요해.', 'prompt': '[INST]user\n옥자 씨, 옥자 씨는 어떻게 하면 좋은 친구가 될 수 있을까요? 혹시 좋은 친구가 되기 위한 옥자 씨만의 노하우가 있으신가요?\nmodel\n좋은 친구? 나는 친구들에게 진심으로 마음을 전하고, 힘이 되어주는 것이 중요하다고 생각해. 그리고 긍정적인 마음으로 친구들을 격려해주고, 함께 웃고 즐거운 시간을 보내는 것도 중요해.[/INST]', 'input_ids': [1, 3, 2956, 781, 1007, 923, 936, 30062, 29473, 1007, 919, 939, 29493, 29473, 1007, 923, 936, 30062, 29473, 1007, 919, 939, 29943, 29473, 30201, 1006, 921, 958, 30601, 29473, 29904, 30333, 29473, 32575, 30306, 29473, 1007, 956, 927, 30547, 29903, 29473, 32473, 29473, 29919, 29473, 30372, 29957, 31346, 30285, 29572, 29473, 1008, 923, 956, 30004, 29473, 32575, 30306, 29473, 1007, 956, 927, 30547, 29903, 29473, 30616, 29932, 29473, 30512, 30050, 29473, 1007, 923, 936, 30062, 29473, 1007, 919, 939, 30525, 29955, 29473, 31293, 29904, 30455, 29903, 294

In [19]:
def get_completion(query: str, model, tokenizer):
  prompt_template = f"""[INST]user\n{query}\nmodel\n"""
  prompt = prompt_template.format(query)
  encoded = tokenizer(prompt, return_tensors = 'pt', add_special_tokens = True)
  model_inputs = encoded.to('cuda:0')
  generated_ids = model.generate(**model_inputs, max_new_tokens = 128)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens = True, padding_side='left')
  return decoded

In [20]:
result = get_completion('오늘하루도 글렀네 간식이나 먹어야지.',model, tokenizer)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


user
오늘하루도 글렀네 간식이나 먹어야지.
model
 뭐 이런 느낌이 들어.













































































































In [ ]:
result

In [28]:
%%time

torch.cuda.empty_cache()

lora_config = LoraConfig(
    r = 32,
    target_modules = ['o_proj','q_proj','up_proj','v_proj','k_proj','down_proj','gate_proj'],
    lora_dropout = 0.05,
    task_type = 'CAUSAL_LM'
)
model = get_peft_model(model, lora_config)

CPU times: user 1.15 s, sys: 17.9 ms, total: 1.17 s
Wall time: 1.17 s


In [ ]:
%%time
trainer = SFTTrainer(
    model = model,
    train_dataset = train_data,
    eval_dataset = eval_data,
    dataset_text_field = 'prompt',
    peft_config = lora_config,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1000,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 5,
        output_dir = 'outputs',
        optim = 'paged_adamw_8bit'
    ),
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:228: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss
5,1.628500
10,1.293200
15,1.022400
20,0.846200
25,0.705000
30,0.581300
35,0.507900
40,0.598700
45,0.479900
50,0.304400


In [ ]:
%%time

result = get_completion('오늘하루도 글렀네 간식이나 먹어야지.',model, tokenizer)
print(result)

# model_save

In [ ]:
!huggingface-cli login

In [ ]:
new_model = 'mistral-7b-v0.3-lora-persona1'
trainer.model.save_pretrained(new_model)
trainer.push_to_hub(f'seriouspark/{new_model}')